In [79]:
import os
import sys
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm

prj_dir = os.path.abspath(os.getcwd() + '/..')
print(prj_dir)
sys.path.append(prj_dir)

/Users/omarolivares/Documents/DataMexico/rulo_mota_omath


In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import shap

from src.features import cnbv_inclusion_financiera
from src.data import banxico_mun_income_remittances

In [3]:
def convert_to_date(x):
    x = str(x)
    return datetime.datetime(int(x[0:4]), int(x[4:]),1)
    

In [4]:
data_paths = dict(
    (x.split('/')[-1], x)
    for x in cnbv_inclusion_financiera.list_datasets())

list(data_paths.keys())

['BD Género Mun.csv',
 'BD Tenencia Uso Banca Mun.csv',
 'BD Infraestructura Mun.csv',
 'BD Tenencia Uso EACP Mun.csv',
 'Municipios.csv',
 'BD Uso Banca Mun.csv',
 'BD Acceso Mun.csv',
 'BD Uso Mun.csv',
 'BD Uso EACP Mun.csv']

In [124]:
data = pd.read_csv(data_paths['BD Tenencia Uso EACP Mun.csv'])
data['date'] = data['date'].apply(convert_to_date)
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year
data = data.fillna(0)
data = data[data['informacion_general__municipio'] !=  'Sin identificar']
data['informacion_general__clave_municipio'] = data['informacion_general__clave_municipio'].astype(int).astype(str)

In [85]:
feature_list = [
       'captacion__depositos_al_ahorro', 
       'captacion__depositos_a_la_vista',
       'captacion__depositos_a_plazo',
       'captacion__tarjetas_de_debito',
       'credito__credito_al_consumo', 
       'credito__credito_a_la_vivienda',
       'transacciones__en_tpv', 
       'transacciones__en_cajeros_automaticos',
       'captacion__depositos_al_ahorro (2)',
       'captacion__depositos_a_la_vista (2)',
       'captacion__depositos_a_plazo (2)',  
       'captacion__tarjetas_de_debito (2)',
       'credito__credito_al_consumo (2)', 
       'credito__credito_a_la_vivienda (2)',
       'transacciones__en_tpv (2)',
       'transacciones__en_cajeros_automaticos (2)',
       'clientes_-_sofipo__mujeres', 'clientes_-_sofipo__hombres',
       'socios_-_socap__mujeres', 'socios_-_socap__hombres',
       'total_de_usuarios_-_eacp__mujeres',
       'total_de_usuarios_-_eacp__hombres', 
       'captacion__deposito_al_ahorro',
       'captacion__deposito_a_la_vista',
       'captacion__deposito_a_plazo', 
       'captacion__tarjeta_debito',
       'captacion__deposito_al_ahorro (2)',
       'captacion__deposito_a_la_vista (2)',
       'captacion__deposito_a_plazo (2)',
       'captacion__tarjeta_debito (2)'
]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7397/7397 [03:48<00:00, 32.35it/s]


In [126]:
X


,mun,year,captacion__depositos_al_ahorro_delta,captacion__depositos_al_ahorro_mean_prop,captacion__depositos_al_ahorro_prop,captacion__depositos_a_la_vista_delta,captacion__depositos_a_la_vista_mean_prop,captacion__depositos_a_la_vista_prop,captacion__depositos_a_plazo_delta,captacion__depositos_a_plazo_mean_prop,...,captacion__deposito_al_ahorro (2)_prop,captacion__deposito_a_la_vista (2)_delta,captacion__deposito_a_la_vista (2)_mean_prop,captacion__deposito_a_la_vista (2)_prop,captacion__deposito_a_plazo (2)_delta,captacion__deposito_a_plazo (2)_mean_prop,captacion__deposito_a_plazo (2)_prop,captacion__tarjeta_debito (2)_delta,captacion__tarjeta_debito (2)_mean_prop,captacion__tarjeta_debito (2)_prop
0,10001,2019,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,2.544993e+03,-447.200421,9.144174e-01,7.113454e-01,38.109253,1.111717e+00,1.342657e+00,21.776716,1.090170,1.285714
0,10001,2020,2725.0,8.903333e+08,2.725000e+09,3067.0,9.816667e+08,3.067000e+09,395.0,1.356667e+08,...,0.000000e+00,-1129.159451,0.000000e+00,0.000000e+00,-145.573763,0.000000e+00,0.000000e+00,-97.049175,0.000000,0.000000
0,10002,2019,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,1.284555e+07,125.244125,4.174804e+07,1.252441e+08,44.959430,1.498648e+07,4.495943e+07,0.000000,0.000000,0.000000
0,10002,2020,7.0,1.333334e+06,7.000000e+06,29.0,1.033333e+07,2.900000e+07,13.0,4.000001e+06,...,0.000000e+00,-101.140195,0.000000e+00,0.000000e+00,-41.088204,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
0,10003,2019,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,1.733333e+01,684.061527,1.876753e+06,1.225000e+02,287.136937,9.571231e+07,2.871369e+08,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,9015,2020,4813.0,1.691000e+09,4.813000e+09,6829.0,2.225667e+09,6.829000e+09,13194.0,5.507333e+09,...,0.000000e+00,-160.417347,0.000000e+00,0.000000e+00,-647.736289,0.000000e+00,0.000000e+00,-160.322552,0.000000,0.000000
0,9016,2019,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,8.928568e+00,53.871257,1.150807e+00,1.503085e+00,-1158.876102,8.671306e-01,6.516032e-01,390.508811,13.184859,90.568161
0,9016,2020,1464.0,3.470000e+08,1.464000e+09,5925.0,1.936667e+09,5.925000e+09,20372.0,9.960333e+09,...,0.000000e+00,-191.320809,0.000000e+00,0.000000e+00,-1709.453085,0.000000e+00,0.000000e+00,-1594.096917,0.000000,0.000000
0,9017,2019,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,1.156226e+00,10.763515,1.044403e+00,1.136538e+00,-187.163883,8.526158e-01,6.197018e-01,0.212238,1.009141,1.027668
